In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv ('kc_house_data.csv')
df = df.drop(columns=['id', 'date', 'zipcode'])
df.head

<bound method NDFrame.head of           price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  \
0      221900.0         3       1.00         1180      5650     1.0   
1      538000.0         3       2.25         2570      7242     2.0   
2      180000.0         2       1.00          770     10000     1.0   
3      604000.0         4       3.00         1960      5000     1.0   
4      510000.0         3       2.00         1680      8080     1.0   
...         ...       ...        ...          ...       ...     ...   
21608  360000.0         3       2.50         1530      1131     3.0   
21609  400000.0         4       2.50         2310      5813     2.0   
21610  402101.0         2       0.75         1020      1350     2.0   
21611  400000.0         3       2.50         1600      2388     2.0   
21612  325000.0         2       0.75         1020      1076     2.0   

       waterfront  view  condition  grade  sqft_above  sqft_basement  \
0               0     0          3      7    

Problem 2.1

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# drop columns
df_train = df_train.drop(columns=['id', 'date', 'zipcode'], errors='ignore')
df_test = df_test.drop(columns=['id', 'date', 'zipcode'], errors='ignore')

# drop unnamed index
df_train = df_train.drop(columns=['Unnamed: 0'], errors='ignore')
df_test = df_test.drop(columns=['Unnamed: 0'], errors='ignore')

# separate features and target
feature_columns = [col for col in df_train.columns if col != 'price']
X_train = df_train[feature_columns].values
y_train = df_train['price'].values

# scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# divide price by 1000
y_train_scaled = y_train / 1000

# train model
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)

# get coefficients
print("intercept (θ₀):", model.intercept_)
print("\ncoefficients:")
for i, feature in enumerate(feature_columns):
    print(f"{feature}: {model.coef_[i]:.4f}")

# predictions on training set
y_train_pred = model.predict(X_train_scaled)

# calculate metrics
train_mse = mean_squared_error(y_train_scaled, y_train_pred)
train_r2 = r2_score(y_train_scaled, y_train_pred)

print(f"\ntraining MSE: {train_mse:.4f}")
print(f"training R²: {train_r2:.4f}")

intercept (θ₀): 520.414834000001

coefficients:
bedrooms: -12.5220
bathrooms: 18.5276
sqft_living: 56.7488
sqft_lot: 10.8819
floors: 8.0437
waterfront: 63.7429
view: 48.2001
condition: 12.9643
grade: 92.2315
sqft_above: 48.2901
sqft_basement: 27.1370
yr_built: -67.6431
yr_renovated: 17.2714
lat: 78.3757
long: -1.0352
sqft_living15: 45.5777
sqft_lot15: -12.9301

training MSE: 31486.1678
training R²: 0.7265


Problem 2.2

In [5]:
# evaluate on test set
X_test = df_test[feature_columns].values
Y_test = df_test['price'].values

# applies transformation
X_test_scaled = scaler.transform(X_test)
Y_test_scaled = Y_test / 1000

# predictiom
Y_test_pred = model.predict(X_test_scaled)

# calculates test metrics
test_mse = mean_squared_error(Y_test_scaled, Y_test_pred)
test_r2 = r2_score(Y_test_scaled, Y_test_pred)

print(f"MSE: {test_mse:.4f}")
print(f"R2: {test_r2:.4f}")

MSE: 57628.1547
R2: 0.6544


Problem 2.3

The features that contributed most are grade, lat, yr_built, and waterfront, given their large coefficients. A variance of 73% shows a good fit, and a 0.65 on test is decent. The average error is about $240,000. Test MSE is almost 2x train MSE, which shows overfitting.

Problem 3

In [9]:
# use the same preprocessed data from problem 2
X_train_bias = np.c_[np.ones(X_train_scaled.shape[0]), X_train_scaled]

# closed-form solution
theta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train_scaled

print("my implementation:")
print(f"intercept: {theta[0]:.4f}")
print("\ncoefficients:")
for i, feature in enumerate(feature_columns):
    print(f"{feature}: {theta[i+1]:.4f}")

# predictions
y_train_pred_manual = X_train_bias @ theta

# metrics
manual_train_mse = mean_squared_error(y_train_scaled, y_train_pred_manual)
manual_train_r2 = r2_score(y_train_scaled, y_train_pred_manual)

print(f"\ntrain MSE: {manual_train_mse:.4f}")
print(f"train R2: {manual_train_r2:.4f}")

# test set
X_test_scaled = scaler.transform(df_test[feature_columns].values)
y_test_scaled = df_test['price'].values / 1000
X_test_bias = np.c_[np.ones(X_test_scaled.shape[0]), X_test_scaled]
y_test_pred_manual = X_test_bias @ theta

manual_test_mse = mean_squared_error(y_test_scaled, y_test_pred_manual)
manual_test_r2 = r2_score(y_test_scaled, y_test_pred_manual)

print(f"\ntest MSE: {manual_test_mse:.4f}")
print(f"test R²: {manual_test_r2:.4f}")

# compare
print("\ncomparison")
print(f"sklearn train MSE: {train_mse:.4f}, my MSE: {manual_train_mse:.4f}")
print(f"sklearn train R²: {train_r2:.4f}, my R²: {manual_train_r2:.4f}")

my implementation:
intercept: 520.4148

coefficients:
bedrooms: 8.5390
bathrooms: 34.9387
sqft_living: -201.7635
sqft_lot: 7.1858
floors: -10.7598
waterfront: 62.0629
view: 55.2758
condition: 14.4307
grade: 87.0247
sqft_above: 271.5224
sqft_basement: 94.3658
yr_built: -67.6431
yr_renovated: 17.2714
lat: 78.3757
long: -1.0352
sqft_living15: 45.5777
sqft_lot15: -12.9301

train MSE: 34322.5211
train R2: 0.7019

test MSE: 62327.2177
test R²: 0.6262

comparison
sklearn train MSE: 31486.1678, my MSE: 34322.5211
sklearn train R²: 0.7265, my R²: 0.7019


Problem 4

In [11]:
from sklearn.preprocessing import StandardScaler

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

scaler = StandardScaler()
X_train = scaler.fit_transform(df_train['sqft_living'].values.reshape(-1, 1))
X_test = scaler.transform(df_test['sqft_living'].values.reshape(-1, 1))

# gets sqft_living feature and price
y_train = df_train['price'].values / 1000
y_test = df_test['price'].values / 1000

# trains different polynomial degrees
degrees = [1, 2, 3, 4, 5]
results = []

# creates polynomial features
for p in degrees:
    X_train_poly = np.ones((X_train.shape[0], 1))
    for power in range(1, p + 1):
        X_train_poly = np.c_[X_train_poly, X_train ** power]

    X_test_poly = np.ones((X_test.shape[0], 1))
    for power in range(1, p + 1):
        X_test_poly = np.c_[X_test_poly, X_test ** power]

    theta = np.linalg.inv(X_train_poly.T @ X_train_poly) @ X_train_poly.T @ y_train

    # predictions
    y_train_pred = X_train_poly @ theta
    y_test_pred = X_test_poly @ theta
    
    # metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    results.append([p, train_mse, train_r2, test_mse, test_r2])

# table
print(f"{'p':<5} {'train MSE':<12} {'train R²':<10} {'test MSE':<12} {'test R²':<10}")
print("-" * 55)
for r in results:
    print(f"{r[0]:<5} {r[1]:<12.4f} {r[2]:<10.4f} {r[3]:<12.4f} {r[4]:<10.4f}")

p     train MSE    train R²   test MSE     test R²   
-------------------------------------------------------
1     57947.5262   0.4967     88575.9785   0.4687    
2     54822.6651   0.5238     71791.6795   0.5694    
3     53785.1947   0.5329     99833.4838   0.4012    
4     52795.7748   0.5415     250979.2743  -0.5053   
5     52626.1120   0.5429     570616.9148  -2.4225   


Problem 5.1

In [13]:
# gradient descent function
def gradient_descent(X, y, learning_rate=0.01, num_iterations=1000):
    n_samples, n_features = X.shape
    theta = np.zeros(n_features)
    
    for i in range(num_iterations):
        # predictions
        y_pred = X @ theta
        
        # gradient
        gradient = (1/n_samples) * X.T @ (y_pred - y)
        
        # update
        theta = theta - learning_rate * gradient
        
        if i % 100 == 0:
            mse = mean_squared_error(y, y_pred)
            print(f"iteration {i}: MSE = {mse:.4f}")
    
    return theta

# use data from problem 3
X_train_bias = np.c_[np.ones(X_train_scaled.shape[0]), X_train_scaled]
X_test_bias = np.c_[np.ones(X_test_scaled.shape[0]), X_test_scaled]

# train with gradient descent
theta_gd = gradient_descent(X_train_bias, y_train_scaled, learning_rate=0.01, num_iterations=1000)

# predictions and metrics
y_train_pred = X_train_bias @ theta_gd
y_test_pred = X_test_bias @ theta_gd

print(f"\ntrain MSE: {mean_squared_error(y_train_scaled, y_train_pred):.4f}")
print(f"train R²: {r2_score(y_train_scaled, y_train_pred):.4f}")
print(f"test MSE: {mean_squared_error(y_test_scaled, y_test_pred):.4f}")
print(f"test R²: {r2_score(y_test_scaled, y_test_pred):.4f}")

iteration 0: MSE = 385968.7732
iteration 100: MSE = 70118.9866
iteration 200: MSE = 36923.1410
iteration 300: MSE = 32390.4390
iteration 400: MSE = 31715.1008
iteration 500: MSE = 31585.8492
iteration 600: MSE = 31544.9992
iteration 700: MSE = 31524.5140
iteration 800: MSE = 31511.9093
iteration 900: MSE = 31503.6358

train MSE: 31498.0867
train R²: 0.7264
test MSE: 57727.6322
test R²: 0.6538


Problem 5.2

In [15]:
# gradient descent with tracking
def gradient_descent_track(X, y, learning_rate, num_iterations):
    n_samples, n_features = X.shape
    theta = np.zeros(n_features)
    
    results = {}
    
    for i in range(1, num_iterations + 1):
        y_pred = X @ theta
        gradient = (1/n_samples) * X.T @ (y_pred - y)
        theta = theta - learning_rate * gradient
        
        if i in [10, 50, 100]:
            results[i] = theta.copy()
    
    return results

# different learning rates
learning_rates = [0.001, 0.01, 0.1]
all_results = []

for alpha in learning_rates:
    theta_dict = gradient_descent_track(X_train_bias, y_train_scaled, alpha, 100)
    
    for iteration in [10, 50, 100]:
        theta = theta_dict[iteration]
        y_train_pred = X_train_bias @ theta
        y_test_pred = X_test_bias @ theta
        
        train_mse = mean_squared_error(y_train_scaled, y_train_pred)
        train_r2 = r2_score(y_train_scaled, y_train_pred)
        test_mse = mean_squared_error(y_test_scaled, y_test_pred)
        test_r2 = r2_score(y_test_scaled, y_test_pred)
        
        all_results.append([alpha, iteration, train_mse, train_r2, test_mse, test_r2])
        print(f"α={alpha}, iter={iteration}: train MSE={train_mse:.4f}, R²={train_r2:.4f}, test MSE={test_mse:.4f}, R²={test_r2:.4f}")

# table
print("\nα      iter   train MSE      train R²     test MSE       test R²")
print("-" * 70)
for r in all_results:
    print(f"{r[0]:<7}{r[1]:<7}{r[2]:<15.4f}{r[3]:<13.4f}{r[4]:<15.4f}{r[5]:.4f}")

α=0.001, iter=10: train MSE=374638.3096, R²=-2.2538, test MSE=446476.9659, R²=-1.6779
α=0.001, iter=50: train MSE=335057.9782, R²=-1.9101, test MSE=398833.3091, R²=-1.3921
α=0.001, iter=100: train MSE=295494.3211, R²=-1.5665, test MSE=351466.1897, R²=-1.1080
α=0.01, iter=10: train MSE=294798.7336, R²=-1.5604, test MSE=350525.0973, R²=-1.1024
α=0.01, iter=50: train MSE=138295.9152, R²=-0.2011, test MSE=170376.6687, R²=-0.0219
α=0.01, iter=100: train MSE=70118.9866, R²=0.3910, test MSE=97486.2448, R²=0.4153
α=0.1, iter=10: train MSE=66499.3155, R²=0.4224, test MSE=93559.2950, R²=0.4388
α=0.1, iter=50: train MSE=31578.9781, R²=0.7257, test MSE=58012.3167, R²=0.6521
α=0.1, iter=100: train MSE=31497.6923, R²=0.7264, test MSE=57725.1857, R²=0.6538

α      iter   train MSE      train R²     test MSE       test R²
----------------------------------------------------------------------
0.001  10     374638.3096    -2.2538      446476.9659    -1.6779
0.001  50     335057.9782    -1.9101      3988

Problem 5.3

The learning rate did affect the convergence speed. With a=0.001, the algorithm learns too slowy, a=0.01, the convergence is moderate and with a=0.1, the convergence is fast achieving R2=0.73 by iteration 100. the number of iterations needed dependend heavily on learning rate. A gradient descent with a=0.1 converges to the same solution as the closed form method from problem 3. This shows that the gradient descent finds the optimal solution when the learning rate is tuned properly. Finally, the MSE values did match closely between both methods.

Problem 6.1

In [19]:
# ridge gradient descent 
def ridge_gradient_descent(X, y, learning_rate, num_iterations, lambda_reg):
    n_samples, n_features = X.shape
    theta = np.zeros(n_features)
    
    for i in range(num_iterations):
        y_pred = X @ theta
        
        theta = theta - learning_rate * gradient
        
        if i % 100 == 0:
            mse = mean_squared_error(y, y_pred)
            print(f"iteration {i}: MSE = {mse:.4f}")
    
    return theta

# test lambda values
lambdas = [0, 0.1, 1.0, 10.0]

for lam in lambdas:
    print(f"\nlambda = {lam}")
    
    theta = ridge_gradient_descent(X_train_bias, y_train_scaled, 0.01, 1000, lam)
    
    y_train_pred = X_train_bias @ theta
    y_test_pred = X_test_bias @ theta
    
    print(f"train MSE: {mean_squared_error(y_train_scaled, y_train_pred):.4f}")
    print(f"train R²: {r2_score(y_train_scaled, y_train_pred):.4f}")
    print(f"test MSE: {mean_squared_error(y_test_scaled, y_test_pred):.4f}")
    print(f"test R²: {r2_score(y_test_scaled, y_test_pred):.4f}")


lambda = 0


NameError: name 'gradient' is not defined

Problem 6.3

In [ ]:
np.random.seed(42)
N = 1000

X = np.random.uniform(-2, 2, N)

noise = np.random.normal(0, np.sqrt(2), N)
Y = 1 + 2*X + noise

# add bias term
X_bias = np.c_[np.ones(N), X]

# closed-form ridge regression function
def ridge_regression(X, y, lambda_reg):
    n_features = X.shape[1]
    I = np.eye(n_features)
    theta = np.linalg.inv(X.T @ X + lambda_reg * I) @ X.T @ y
    return theta

# test lambda values
lambdas = [0, 1, 10, 100, 1000, 10000]
results = []

print("λ        intercept   slope      MSE        R²")
print("-" * 60)

for lam in lambdas:
    theta = ridge_regression(X_bias, Y, lam)
    
    Y_pred = X_bias @ theta
    
    mse = mean_squared_error(Y, Y_pred)
    r2 = r2_score(Y, Y_pred)
    
    results.append([lam, theta[0], theta[1], mse, r2])
    print(f"{lam:<8} {theta[0]:<11.4f} {theta[1]:<10.4f} {mse:<10.4f} {r2:.4f}")

print("\ntrue values: intercept = 1.0, slope = 2.0")